In [1]:
# Import necessary libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
from sqlalchemy import create_engine

In [2]:
base_url = 'https://www.kaiandkaro.com/vehicles?model__make__vehicle_type=Automobile'
response = requests.get(base_url)
soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
# Get the last element from the pagination list
page_links = soup.find_all('li', class_ = 'chakra-wrap__listitem css-1yp4ln')
max_page = int(page_links[-1].text)
print(f"Scraping {max_page} pages...")

Scraping 99 pages...


In [38]:
car_list = []
for page_num in range(1, max_page + 1):
    url = f"{base_url}&page={page_num}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    car_details = soup.find_all('div', class_ = 'chakra-linkbox css-1kh7oap')
    for car_detail in car_details:
        try:
            name = car_detail.find('h2', class_='chakra-heading css-18j379d').text.strip()
        except:
            name = " "
        try:
            transmission_type = car_detail.find('span', class_='css-evl6jo').text.strip()
        except:
            transmission_type = " "
        try:
            engine_size = car_detail.find_all('span', class_='css-evl6jo')[1].text.strip()
        except:
            engine_size = " "
        try:
            usage_origin = car_detail.find('span', class_='css-b03jaa').text.strip()
        except:
            usage_origin = " "
        try:
            year = car_detail.find('span', class_='chakra-badge css-1dub5x4').text.strip()
        except:
            year = " "
        try:
            price = car_detail.find('p', class_='chakra-text css-0').text.strip()
        except:
            price = " "

        car_dict = {'Name':name,
                    'Transimission_Type': transmission_type,
                    'Engine_Size': engine_size,
                    'Usage_Origin': usage_origin,
                    'Year': year,
                    'Price': price}
        car_list.append(car_dict)

car_df = pd.DataFrame(car_list)
car_df

,Name,Transimission_Type,Engine_Size,Usage_Origin,Year,Price
0,Toyota Harrier,Automatic,2000 CC,Foreign Used,2017,"KES 3,450,000"
1,Audi SQ5,Automatic,3000 CC,Foreign Used,2018,"KES 6,463,700"
2,Range Rover Vogue SV,Automatic,4400 CC,Foreign Used,2023,"KES 51,400,000"
3,Isuzu MUX,Automatic,3000 CC,Foreign Used,2022,"KES 8,600,000"
4,Mazda CX5,Automatic,2500 CC,Foreign Used,2019,"KES 4,788,209"
...,...,...,...,...,...,...
2440,Toyota Auris,Automatic,1500 CC,Foreign Used,2016,"KES 1,800,000"
2441,Peugeot 207,Automatic,1600 CC,Kenyan Used,2012,"KES 950,000"
2442,Mazda Atenza,Automatic,2200 CC,Foreign Used,2016,"KES 2,150,000"
2443,Isuzu Elf Light Truck,Automatic,Foreign Used,Foreign Used,2014,"KES 1,850,000"


In [43]:
car_df.to_csv('car_df.csv', index = False)

In [40]:
# Database credentials
db_user = 'root' 
db_password = 'password'
db_host = 'localhost'
db_name = 'car_database'

In [41]:
# Create a connection to MySQL database
engine = create_engine(f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}')

In [42]:
# Save the dataframe to mysql
table_name = 'car_database'
car_df.to_sql(name = table_name, con = engine, if_exists = 'replace', index = False)

2445